В этой практике вы познакомитесь с тем как строить пайплайн для решения задачи обучения с учителем. На самом деле строить пайплайн с нуля даже для такой задачи как классификация кошек и собак довольно трудоемко для первой практики, поэтому многие шаги я уже приготовил. Вам нужно реализовать ключевые куски чтобы сложить картину целиком.
Для того чтобы выполнить практику, поставьте [Anaconda](https://www.anaconda.com/download) - она содержит нужные для работы библиотеки.

Зарегистрируйтесь на платформе https://kaggle.com и посмотрите конкурс https://www.kaggle.com/c/dogs-vs-cats

Скачайте данные этого конкурса https://www.kaggle.com/c/3362/download-all. Наш проект будет построен по т.н. схеме [cookiecutter](https://drivendata.github.io/cookiecutter-data-science/#cookiecutter-data-science) (прочитать про неё полезно, но не обязательно), поэтому скачанные данные распакуйте и поместите в папку `data/` внутри проекта (на деле если вы используете ссылку выше нужно распаковать дважды - архив и архивы внутри архива). Если вы используете `git`, то добавьте папку `data/` в файл `.gitignore`.

Данные для каждой практики персональны для каждого студента. Введите свое имя, фамилию и группу в строковую переменную STUDENT_ID (в произвольной форме) - это ваш уникальный идентификатор.

In [23]:
STUDENT_ID = "Radyushin_Vadim_411"

Чтение данных
====

Обработаем данные. Этот шаг я сделаю за вас в силу некоторой его трудоемкости. Используем библиотеки pandas, numpy и Pillow.
- Введение в pandas и numpy можно посмотреть в первом уроке курса mlcourse_open: 
- https://github.com/Yorko/mlcourse.ai/blob/master/jupyter_russian/topic01_pandas_data_analysis/lesson1_part0_numpy.ipynb
- https://github.com/Yorko/mlcourse.ai/blob/master/jupyter_russian/topic01_pandas_data_analysis/lesson1_part1_pandas_intro.ipynb
- Pillow - одна из самых распространенных библиотек для работы с изображениями в python.

In [24]:
import pandas as pd
import pathlib
import hashlib
import numpy as np
import random


train_directory = pathlib.Path("data/train")
sample_size = 5000

def initialize_random_seed():
    """Инициализирует ГПСЧ из STUDENT_ID"""
    sha256 = hashlib.sha256()
    sha256.update(STUDENT_ID.encode("utf-8"))
    
    fingerprint = int(sha256.hexdigest(), 16) % (2**32) 
    
    random.seed(fingerprint)
    np.random.seed(fingerprint)


def read_target_variable():
    """Прочитаем разметку фотографий из названий файлов"""
    target_variable = {
        "filename": [],
        "is_cat": []
    }
    image_paths = list(train_directory.glob("*.jpg"))
    random.shuffle(image_paths)
    for image_path in image_paths[:sample_size]:
        filename = image_path.name
        class_name = filename.split(".")[0]
        target_variable["filename"].append(filename)
        target_variable["is_cat"].append(class_name == "cat")

    return pd.DataFrame(data=target_variable)


initialize_random_seed()

target_df = read_target_variable()
target_df

,filename,is_cat
0,dog.1182.jpg,False
1,cat.4963.jpg,True
2,dog.9375.jpg,False
3,dog.12.jpg,False
4,dog.11321.jpg,False
...,...,...
4995,cat.1182.jpg,True
4996,dog.4231.jpg,False
4997,dog.4653.jpg,False
4998,dog.11488.jpg,False


Обработка изображений
====

Используем изображения в качестве признакового описания. Признаками будут значения конкретных пикселей.

In [25]:
from PIL import Image
from tqdm import tqdm_notebook


def read_data(target_df):
    """Читает данные изображений и строит их признаковое описание"""
    image_size = (100, 100)
    features = []
    target = []
    for i, image_name, is_cat in tqdm_notebook(target_df.itertuples(), total=len(target_df)):
        image_path = str(train_directory / image_name)
        image = Image.open(image_path)
        image = image.resize(image_size) # уменьшаем изображения
        image = image.convert('LA') # преобразуем в Ч\Б
        pixels = np.asarray(image)[:, :, 0]
        pixels = pixels.flatten()
        features.append(pixels)
        target.append(is_cat)
    return np.array(features), np.array(target)

features, target = read_data(target_df)
features

  0%|          | 0/5000 [00:00<?, ?it/s]

array([[ 74,  73,  70, ...,  56,  61,  83],
       [106, 109, 112, ..., 106,  97,  99],
       [196, 195, 194, ..., 137, 111,  97],
       ...,
       [195, 193, 168, ...,  31,  34,  37],
       [ 84,  70,  72, ..., 100,  99, 110],
       [ 78,  80,  81, ...,  99,  98,  98]], dtype=uint8)

Разбиение на обучающую и тестовую выборки
====

Сделайте обучающую и тестовую выборки в пропорции 80/20. Используйте библиотеку scikit-learn (метод train_test_split). 

Документация: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Советую изучить готовый пайплайн для другой задачи, чтобы примерно понимать что требуется делать дальше (в отличии от нашего пайплайна тут нет валидационного множества): https://scikit-learn.org/stable/auto_examples/linear_model/plot_sparse_logistic_regression_mnist.html 


In [26]:
from sklearn.model_selection import train_test_split

features_train, features_test, target_train, target_test = train_test_split(features, target, train_size=0.8, test_size=0.2, random_state=42)
features_train

array([[145, 159, 169, ...,  41,  40,  41],
       [ 92, 109, 112, ..., 249, 250, 250],
       [ 14,  16,  20, ..., 202, 209, 211],
       ...,
       [108, 116, 138, ...,  53,  49,  40],
       [141, 140, 140, ..., 121, 121, 120],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

Обучение модели
====

Попробуйте обучить модель `sklearn.linear_model.SGDClassifier` на полученных данных. Если это занимает слишком много времени и ресурсов - уменьшайте размер выборки (переменная sample_size).

Документация: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html

In [27]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, eta0=0.5, learning_rate="constant")
sgd_clf.fit(features_train, target_train)

SGDClassifier(eta0=0.5, learning_rate='constant')

Настройка гиперпараметров
====

Подберите скорость обучения модели, оценивая долю верных ответов (scikit-learn accuracy_score) на валидационном множестве (подбираем параметр `eta0` в отрезке [0, 1] указав параметр `learning_rate="constant"`).

Оценка результатов
====

Оцените долю верных ответов модели (scikit-learn accuracy_score). Каков уровень переобучения?

In [28]:
from sklearn.metrics import accuracy_score

accuracy_score(target_test, clf.predict(features_test))

0.52

Нормализация данных
====

Сделайте нормализацию данных с помощью StandardScaler из библиотеки sklearn (scaler.fit_transform на train и validation, scaler.transform на test). Оцените долю верных ответов модели на тестовой выборке. Какова ситуация теперь?

Документация: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

scaler = StandardScaler()

features_train_normalized = scaler.fit_transform(features_train)
features_test_normalized = scaler.transform(features_test)

sgd_clf.fit(features_train_normalized, target_train)

accuracy_score(target_test, clf.predict(features_test_normalized))


0.523

Сабмит на kaggle
====

(Опционально) сделайте сабмит в конкурсе https://www.kaggle.com/c/dogs-vs-cats (это простой бейзлайн, не ожидайте высокого результата)

# Random Forest

Примените RandomForest вместо SGDClassifier на этой задаче. Подберите количество деревьев, перебирая их в интервале от 100 до 400.


In [30]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=300, max_depth=5, random_state=42)

rf_clf.fit(features_train_normalized, target_train)

accuracy_score(target_test, rf_clf.predict(features_test_normalized))

0.61

Оцените качество RandomForest на кросс-валидации по пяти фолдам. Посмотрите на среднее и стандартное отклонение точности на кросс-валидации.

In [31]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf_clf, features, target, cv=5)
scores

array([0.614, 0.597, 0.616, 0.608, 0.607])

In [32]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.61 accuracy with a standard deviation of 0.01
